In [ ]:
import psi4
import fortecubeview as vis
import numpy as np

# Problem 3: Calculating vibrational frequencies
<span style="color:red">WARNING: Any modifications you make here will not be saved! Make sure you download what you need before closing the tab. </span>

The code below is from the Psi4 demo. We are going to calculate the vibrational frequencies of a benzene molecule starting from Cartesian force constant maxtrix (Hessian).

In [ ]:
Benzene = psi4.geometry("""pubchem:Benzene""")

psi4.set_options({'basis': '6-31g',
                  'reference': 'rhf'})
psi4.core.set_output_file('vib.out')
#psi4.optimize('scf', molecule = Benzene)

psi4.set_options({"normal_modes_write ":True,
                  "writer_file_label":"C6H6"})

#E, wfn = psi4.frequency('scf', molecule=Benzene, return_wfn=True)

# I commented out the psi4.optimize() and psi4.frequency() and provided its normal mode vibrational frequencies.
# This is because the Binder server won't let us run anything too expensive, such as calculating Hessian of a benzene molecule.
# If you want to calculate the frequencies by yourself, download this script and run it with your own computer. 


freqs = np.loadtxt("Benzene_freqs.txt", dtype=float)
print("\nWe want to get these numbers from Cartesian coordinate based Hessian \n", freqs)

### Problem 3-1: Do you see the two lowest vibrational frequencies are very close to each other? Visualize them and explain why they are so close to each other.

In [ ]:
vis.vib("C6H6.molden_normal_modes")

### Problem 3-2: Hessians can be obtained by using `psi4.hessian()`. Follow the comments to calculate vibrational frequencies from the Hessian. Submit your code inside of the 'Your code goes here' block (screenshot works) and the printed result.
Let's discuss units before we start coding. The unit of an Hessian element is $E_h/Bohr^2$ where $E_h$ is the Hartree energy. First, we need to get the mass-weighed Hessian matrix.
<p style="text-align: center;">$H'_{j,k} = \dfrac{H_{j,k}}{ \sqrt{m_jm_k} }$</p> 

The `mass_array` we get from `M.to_dict()['mass']` below has the unit of atomic mass. Since one of the Hartree atomic units is $m_e$ (mass of electron), we need to convert the atomic mass to the mass of electron ($m_p/m_e$ = 1836.153). \
The length of `mass_array` is same as the total number of atoms (N). However, the Hessian has a length of 3N (x, y, and z for each atom). We can use `np.repeat()` to fix that and diagonalize the mass matrix. \
Once we have the diagonalized mass matrix `M`, we can get the mass-weighted Hessian by:

<p style="text-align: center;">$ H' = H_{mw} =  M^{-1/2}  H  M^{-1/2}$</p> 

Note that you won't be able to inverse `M` matrix because you have zero off-diagonal elements. You need to inverse the mass values before diagonalize it. \
Once we have the `H_mw`, we can calculate it's eigenvalues using `lambdas, vectors = np.linalg.eig(H_mw)`. Each eigenvalue in the `lambdas` represents force constant ($k'$) of a normal vibraional mode. 

<p style="text-align: center;">$ \nu * 2\pi = \omega = \sqrt{k'}$</p>
<p style="text-align: center;">$ \nu = \omega/2\pi$</p>

The $k'$ has a unit of $\dfrac{E_h}{Bohr^2 m_e}$. We can convert the mass to energy by using $E_h = m_ec^2$, then the unit becomes $\dfrac{c^2}{Bohr^2}$. \
Now the $\omega = \sqrt{k'}$ has a unit of $\dfrac{c}{Bohr}$ and we can remove the speed of light by using its value in atomic units. The value is inverse of the fine-structure constant ($\alpha = {e^2}/{(4\pi\epsilon_0)\hbar c} \approx 1/137$) which is $c \approx 137$. \
Finally, we can convert the $Bohr$ to $cm^{-1}$ ($1 Bohr = 5.2918 * 10^{-9} cm$). 

In [ ]:
na = len(Benzene.geometry().np[:]) # Total number of atoms

#H, wfn = psi4.hessian('scf', molecule = Benzene,return_wfn = True)

H = np.loadtxt("Benzene_Hessian.txt", dtype=float)

###Your code goes here
mass_array = Benzene.to_dict()['mass'] # A mass array with a length of N of all atoms.
mass_array = # Here you want to convert the atomic mass unit to electronic mass. Don't forget to inverse them.  
m = np.repeat() # Here you want to repeat the mass_array it 3 times, so the new array covers all coordinates.
M = np.diag(m) 
print(M) # Make sure this matrix makes sense to you

H_mw = np.dot(np.dot(M,H),M) # Getting the mass-weighted Hessian
lambdas, vectors = np.linalg.eig(H_mw) # np.linalg.eig() is a convenient function that can calculate eigenvalues and their vectors.
f_const = lambdas[:na*3-6] # We are picking eigenvalues that are not close to 0 (3N-6 of them since Benzene is not a linear molecule)

freq = np.sqrt(f_const)/(2*np.pi) # You need to convert Bohr to cm-1 and use the speed of light in atomic units here.

print(np.sort(freq)) # Sorted to match the order above.
###
